In [1]:
!pip install transformers datasets scikit-learn pandas tqdm
!pip install beautifulsoup4
!pip install unidecode
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.3 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import re
from bs4 import BeautifulSoup
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
# Load dataset (adjust the path if needed)
df = pd.read_csv("/content/twitter_training.csv", header=None, names=["ID", "Topic", "Sentiment", "Text"])
df_test = pd.read_csv("/content/twitter_validation.csv", header=None, names=["ID", "Topic", "Sentiment", "Text"])

# Filter necessary columns
df = df[["Sentiment", "Text"]]
df_test = df_test[["Sentiment", "Text"]]

In [4]:
# df = df.sample(frac=0.6, random_state=42)

In [5]:
!nvidia-smi

Mon Jun  9 03:28:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
# Label encoding
label2id = {"Positive": 0, "Negative": 1, "Neutral": 2, "Irrelevant": 3}
id2label = {v: k for k, v in label2id.items()}
df = df[df["Sentiment"].isin(label2id)]  # filter out undefined labels
df["Label"] = df["Sentiment"].map(label2id)
df_test["Label"] = df_test["Sentiment"].map(label2id)

### DistilBert

In [13]:
# Preprocessing function
# Ensure no NaNs before cleaning
df["Text"] = df["Text"].fillna("")
df_test["Text"] = df_test["Text"].fillna("")

# Set of stopwords
stop_words = set(stopwords.words('english'))

# Text cleaning function
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()               # Remove HTML tags
    text = re.sub(r"http\S+|www\S+|pic\.twitter\S+", "", text)        # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)                            # Remove special chars/emojis
    text = text.lower()                                               # Lowercase
    text = unidecode(text)                                            # Normalize Unicode
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]        # Remove stopwords
    return " ".join(words)

# Apply cleaning
df['clean_text'] = df['Text'].apply(clean_text)
df_test['clean_text'] = df_test['Text'].apply(clean_text)

# Tokenization
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.long)
        }

<ipython-input-13-b812a7055330>:11: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html.parser").get_text()               # Remove HTML tags


In [14]:
train_texts = df["Text"].values
train_labels = df["Label"].values

test_texts = df_test["Text"].values
test_labels = df_test["Label"].values

# Tokenize and create datasets
train_dataset = TweetDataset(train_texts, train_labels, tokenizer)
test_dataset = TweetDataset(test_texts, test_labels, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [15]:
# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
model.to(device)

# Training setup
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Learning rate Scheduling
epochs = 5
num_training_steps = len(train_loader) * epochs
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [20]:
# Training loop
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Gradient Clipping
        optimizer.step()
        scheduler.step() # Finetuning strategy 1: Warmup Scheduler
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

Epoch 5: 100%|██████████| 4668/4668 [07:56<00:00,  9.80it/s, loss=0.0013]


In [22]:
# Evaluation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification metrics
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=list(label2id.keys())))



Classification Report:
              precision    recall  f1-score   support

    Positive       0.94      0.98      0.96       277
    Negative       0.99      0.97      0.98       266
     Neutral       0.96      0.96      0.96       285
  Irrelevant       0.98      0.94      0.96       172

    accuracy                           0.96      1000
   macro avg       0.97      0.96      0.96      1000
weighted avg       0.97      0.96      0.97      1000



### Bert

In [ ]:
# Preprocessing function
# Ensure no NaNs before cleaning
df["Text"] = df["Text"].fillna("")
df_test["Text"] = df_test["Text"].fillna("")

# Set of stopwords
stop_words = set(stopwords.words('english'))

# Text cleaning function
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()               # Remove HTML tags
    text = re.sub(r"http\S+|www\S+|pic\.twitter\S+", "", text)        # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)                            # Remove special chars/emojis
    text = text.lower()                                               # Lowercase
    text = unidecode(text)                                            # Normalize Unicode
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]        # Remove stopwords
    return " ".join(words)

# Apply cleaning
df['clean_text'] = df['Text'].apply(clean_text)
df_test['clean_text'] = df_test['Text'].apply(clean_text)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.long)
        }

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
model.to(device)

# Training setup
optimizer = AdamW(model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training loop
epochs = 3
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

Epoch 3: 100%|██████████| 4668/4668 [15:20<00:00,  5.07it/s, loss=0.0744]


In [ ]:
# Evaluation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification metrics
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=list(label2id.keys())))


Classification Report:
              precision    recall  f1-score   support

    Positive       0.90      1.00      0.95       277
    Negative       0.98      0.97      0.98       266
     Neutral       0.99      0.90      0.94       285
  Irrelevant       0.96      0.96      0.96       172

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000

